In [7]:
import pandas as pd
from binance.client import Client
from Binance_api import api_key, secret_key
from datetime import datetime
from datetime import timedelta
import time
from binance.exceptions import BinanceAPIException
import pandas as pd
import numpy as np
import os
from binance.client import Client
from Binance_api import api_key, secret_key
import time
import plotly.graph_objects as go
import talib as ta
from datetime import datetime
import os
from ta import add_all_ta_features
import warnings
import sklearn
from sklearn.decomposition import PCA
import plotly.express as px
import numpy
import plot_utils 
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
from knockknock import email_sender

warnings.filterwarnings('ignore')

client = Client(api_key, secret_key,tld='us')

In [8]:
def sma(data, window):
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band


In [9]:
symbols = ['BTC']
start_n_hours_ago = 10
def gather_data(symbols, start_n_hours_ago):
        merge = False
        for symbol in symbols:
                klines = client.get_historical_klines(symbol=f'{symbol}USDT', interval=client.KLINE_INTERVAL_5MINUTE, start_str=str(datetime.now()-timedelta(hours=start_n_hours_ago)))
                
                cols = ['OpenTime',
                        f'{symbol}-USD_Open',
                        f'{symbol}-USD_High',
                        f'{symbol}-USD_Low',
                        f'{symbol}-USD_Close',
                        f'{symbol}-USD_Volume',
                        'CloseTime',
                        f'{symbol}-QuoteAssetVolume',
                        f'{symbol}-NumberOfTrades',
                        f'{symbol}-TBBAV',
                        f'{symbol}-TBQAV',
                        f'{symbol}-ignore']
                
                df = pd.DataFrame(klines,columns=cols)

                if merge == True:
                        dfs = pd.merge(df,dfs,how='inner',on=['OpenTime','CloseTime'])
                else:
                        dfs = df
                        merge = True
        dfs['OpenTime'] = [datetime.fromtimestamp(ts / 1000) for ts in dfs['OpenTime']]
        dfs['CloseTime'] = [datetime.fromtimestamp(ts / 1000) for ts in dfs['CloseTime']]

        for col in dfs.columns: 
                if not 'Time' in col:
                        dfs[col] = dfs[col].astype(float)

        for symbol in symbols:
                dfs[f'{symbol}_sma'] = sma(dfs[f'{symbol}-USD_Close'],window=20)
                dfs[f'{symbol}_upper_band'], dfs[f'{symbol}_lower_band'] = bollinger_band(data=dfs[f'{symbol}-USD_Close'], sma=dfs[f'{symbol}_sma'], window=20, nstd=3)
                #dfs[f'{symbol}_rsi'] = RSIIndicator(dfs[f'{symbol}-USD_Close'], window = 14, fillna = False)
                dfs[f'{symbol}_rsi'] = ta.RSI(dfs[f'{symbol}-USD_Close'])

        dfs.dropna(inplace=True)

        return dfs

def get_states(df, symbols):
    states = {}

    for symbol in symbols:
        if df[f'{symbol}-USD_Close'].iloc[-1] < df[f'{symbol}_lower_band'].iloc[-1]:
            states[symbol] = 'below'
        elif df[f'{symbol}-USD_Close'].iloc[-1] > df[f'{symbol}_upper_band'].iloc[-1]:
            states[symbol] = 'above'
        else:
            states[symbol] = 'inside'


In [10]:
df = gather_data(symbols,48)


In [11]:
df


,OpenTime,BTC-USD_Open,BTC-USD_High,BTC-USD_Low,BTC-USD_Close,BTC-USD_Volume,CloseTime,BTC-QuoteAssetVolume,BTC-NumberOfTrades,BTC-TBBAV,BTC-TBQAV,BTC-ignore,BTC_sma,BTC_upper_band,BTC_lower_band,BTC_rsi
19,2022-11-27 04:55:00,16482.65,16482.65,16472.17,16472.17,0.141754,2022-11-27 04:59:59.999,2335.778836,8.0,0.024046,396.310783,0.0,16510.8825,16556.366994,16465.398006,30.005904
20,2022-11-27 05:00:00,16463.56,16463.56,16427.15,16458.34,12.948552,2022-11-27 05:04:59.999,212934.021967,99.0,8.686814,142875.805736,0.0,16507.3960,16563.265990,16451.526010,26.589406
21,2022-11-27 05:05:00,16452.66,16461.20,16452.66,16459.96,0.231438,2022-11-27 05:09:59.999,3809.479037,5.0,0.000000,0.000000,0.0,16504.1940,16567.119388,16441.268612,27.628889
22,2022-11-27 05:10:00,16465.75,16472.73,16451.91,16460.08,4.016211,2022-11-27 05:14:59.999,66113.912733,40.0,1.270220,20910.360294,0.0,16501.3515,16570.112212,16432.590788,27.710544
23,2022-11-27 05:15:00,16454.87,16464.75,16454.87,16463.61,0.682215,2022-11-27 05:19:59.999,11231.548655,13.0,0.634123,10440.145515,0.0,16499.0260,16571.930787,16426.121213,30.205253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,2022-11-28 16:55:00,16207.21,16218.62,16206.32,16212.23,0.244382,2022-11-28 16:59:59.999,3962.226779,24.0,0.244382,3962.226779,0.0,16180.5315,16230.917435,16130.145565,58.057983
452,2022-11-28 17:00:00,16215.67,16216.74,16205.58,16205.58,0.263701,2022-11-28 17:04:59.999,4274.136443,27.0,0.239810,3886.957006,0.0,16181.6580,16234.770318,16128.545682,55.480774
453,2022-11-28 17:05:00,16207.08,16209.24,16200.52,16206.03,0.177925,2022-11-28 17:09:59.999,2883.214627,20.0,0.116170,1882.742914,0.0,16183.1830,16238.522532,16127.843468,55.624326
454,2022-11-28 17:10:00,16202.63,16204.34,16191.30,16199.06,2.699768,2022-11-28 17:14:59.999,43721.867037,49.0,0.476838,7725.486040,0.0,16183.2865,16238.887474,16127.685526,52.785243



states

In [12]:
balance_unit = 'USDT'

First = True 
while True:
    if (datetime.now().second % 10 == 0) or First:
        if (datetime.now().minute == 0 and datetime.now().second == 10) or First:
            #refresh data
            first = False
            df = gather_data(symbols,48)
            states = get_states(df, symbols)
            print('current state of the market:')
            print(states)
    try:
        print('\n')
        if balance_unit == 'USDT': #looking to buy
            for symbol in symbols:
                ask_price = float(client.get_orderbook_ticker(symbol=f'{symbol}USDT')['askPrice'])
                lower_band = df[f'{symbol}_lower_band'].iloc[-1]
                print(f'{symbol}: ask price {ask_price} | lower band {lower_band}')
                if ask_price < lower_band and states[symbol] == 'inside':
                    print('buy')
                    balance_unit = symbol
                    break

        if balance_unit != 'USDT': #looking to sell
            for symbol in symbols:
                bid_price = float(client.get_orderbook_ticker(symbol= f'{balance_unit}USDT')['bidPrice'])
                upper_band = df[f'{symbol}_upper_band'].iloc[-1]
                if bid_price > upper_band and states[symbol] == 'inside':
                    print('symbol')
                    balance_unit = 'USDT'
                    break
        time.sleep(1)

    except BinanceAPIException as e:
        print(e.status_code)
        print(e.message)

        

current state of the market:
None


BTC: ask price 16199.06 | lower band 16127.452897141677
current state of the market:
None


BTC: ask price 16200.17 | lower band 16127.452897141677
current state of the market:
None


BTC: ask price 16200.17 | lower band 16127.452897141677
current state of the market:
None


BTC: ask price 16200.17 | lower band 16127.338319934586
current state of the market:
None


BTC: ask price 16200.17 | lower band 16127.338319934586
current state of the market:
None


BTC: ask price 16200.17 | lower band 16127.338319934586
current state of the market:
None


BTC: ask price 16200.17 | lower band 16127.338319934586
current state of the market:
None


BTC: ask price 16200.17 | lower band 16127.338319934586
current state of the market:
None


BTC: ask price 16200.17 | lower band 16127.338319934586
current state of the market:
None


BTC: ask price 16200.17 | lower band 16127.338319934586
current state of the market:
None


BTC: ask price 16200.17 | lower band 16127.3

KeyboardInterrupt: 

,OpenTime,ETH-USD_Open,ETH-USD_High,ETH-USD_Low,ETH-USD_Close,ETH-USD_Volume,CloseTime,ETH-QuoteAssetVolume,ETH-NumberOfTrades,ETH-TBBAV,...,BTC-NumberOfTrades,BTC-TBBAV,BTC-TBQAV,BTC-ignore,BTC_sma,BTC_upper_band,BTC_lower_band,ETH_sma,ETH_upper_band,ETH_lower_band
19,2022-11-25 19:00:00,1180.29,1183.65,1177.08,1177.08,7.86981,2022-11-25 19:59:59.999,9287.445698,27.0,6.15863,...,320.0,18.483206,304335.491921,0.0,16520.6305,16746.054491,16295.206509,1192.0910,1221.688293,1162.493707
20,2022-11-25 20:00:00,1177.34,1188.63,1177.16,1185.02,15.55333,2022-11-25 20:59:59.999,18426.669119,53.0,11.14847,...,436.0,15.589596,256783.317860,0.0,16517.5975,16744.855139,16290.339861,1191.6310,1221.556438,1161.705562
21,2022-11-25 21:00:00,1188.33,1198.36,1188.33,1193.58,90.16966,2022-11-25 21:59:59.999,107667.104653,146.0,44.57736,...,881.0,41.795450,691001.318543,0.0,16516.2835,16742.019856,16290.547144,1191.5560,1221.416394,1161.695606
22,2022-11-25 22:00:00,1192.51,1199.90,1191.33,1191.33,4.41707,2022-11-25 22:59:59.999,5282.181152,44.0,2.91564,...,760.0,30.260903,500682.454559,0.0,16510.4180,16723.236793,16297.599207,1191.0430,1220.051327,1162.034673
23,2022-11-25 23:00:00,1191.24,1192.62,1187.45,1190.01,16.53533,2022-11-25 23:59:59.999,19683.934642,33.0,7.38857,...,311.0,14.882641,245604.225225,0.0,16505.6205,16711.669390,16299.571610,1190.5110,1218.717184,1162.304816
24,2022-11-26 00:00:00,1189.48,1193.39,1181.92,1191.54,62.42246,2022-11-26 00:59:59.999,74020.434152,126.0,28.44296,...,555.0,30.292847,499274.361662,0.0,16502.9655,16702.462798,16303.468202,1189.8745,1216.380708,1163.368292
25,2022-11-26 01:00:00,1193.82,1195.58,1187.75,1189.94,35.10816,2022-11-26 01:59:59.999,41787.516215,74.0,11.40928,...,637.0,30.359787,501619.981394,0.0,16499.8405,16693.046334,16306.634666,1189.2145,1214.015680,1164.413320
26,2022-11-26 02:00:00,1188.91,1190.22,1183.79,1186.66,31.31076,2022-11-26 02:59:59.999,37132.615612,50.0,25.60528,...,426.0,20.433392,336805.757061,0.0,16495.8475,16683.047822,16308.647178,1188.6000,1212.468732,1164.731268
27,2022-11-26 03:00:00,1187.91,1194.07,1187.91,1191.47,49.67030,2022-11-26 03:59:59.999,59209.903942,97.0,23.62098,...,559.0,29.360904,484446.925628,0.0,16493.4180,16675.862158,16310.973842,1188.3510,1211.671401,1165.030599
28,2022-11-26 04:00:00,1190.61,1193.83,1189.18,1189.18,24.53324,2022-11-26 04:59:59.999,29230.836105,37.0,15.91869,...,274.0,7.402413,122178.696092,0.0,16487.2885,16647.809215,16326.767785,1187.6180,1208.244085,1166.991915


In [46]:
try:
    client.get_all_orders()
except BinanceAPIException as e:
    print(e.status_code)
    print(e.message)


NameError: name 'BinanceAPIException' is not defined

In [11]:
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=0.99925, symbols=symbols)

for i in range(len(df)):  
    #means looking for something to buy
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if env.bottoms[symbol] == 'hit' and df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]:
                env.bottoms[symbol] = 'released'
            
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]: #buy signal
                if env.bottoms[symbol] == 'released':
                    env.buy(symbol, df[f'{symbol}_lower_band'].iloc[i], df['OpenTime'].iloc[i])
                    env.reset_bottoms()
                    break
                else:
                    env.bottoms[symbol] = 'hit'
    
    #means looking to sell
    if env.balance_unit != 'USDT':
        if env.tops[env.balance_unit] == 'hit' and df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i]:
            env.tops[env.balance_unit] = 'released'
        
        if df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i]: #sell signal
            if env.tops[env.balance_unit] == 'released':
                env.sell(df[f'{env.balance_unit}_upper_band'].iloc[i], df['OpenTime'].iloc[i])
                env.reset_tops()
            else:
                env.tops[env.balance_unit] = 'hit'

if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])
    
   

In [12]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')

num buys: 70
num sells: 70
ending balance: 89.33740476289911 USDT


In [31]:
for i in range(len(env.buys)):
    buys = env.buys[i]
    sells = env.sells[i]
    profit = ((sells[2]/buys[2])-1)*100
    print(buys)
    print(sells)
    print('P/L = 'f'{profit}')
    print('\n')

['BTC', '2022-11-16 10:56:00', 16792.25083691415]
['BTC', '2022-11-16 11:57:00', 16855.338406492894]
P/L = 0.37569454024626303


['BTC', '2022-11-16 14:22:00', 16928.967577869214]
['BTC', '2022-11-16 15:18:00', 16922.039155381153]
P/L = -0.0409264324962022


['BTC', '2022-11-16 16:11:00', 16889.292258865134]
['BTC', '2022-11-16 17:02:00', 16896.161880173106]
P/L = 0.04067441786594905


['BTC', '2022-11-16 18:00:00', 16815.619922646805]
['BTC', '2022-11-16 20:04:00', 16773.04289935849]
P/L = -0.2531992485806267


['BTC', '2022-11-16 20:36:00', 16691.752089485755]
['BTC', '2022-11-16 20:43:00', 16754.65165393053]
P/L = 0.37683020995977756


['BTC', '2022-11-16 21:49:00', 16709.191793541515]
['BTC', '2022-11-16 23:33:00', 16674.15172413135]
P/L = -0.20970535165986126


['BTC', '2022-11-16 23:56:00', 16474.124285860224]
['BTC', '2022-11-17 00:34:00', 16578.37796267964]
P/L = 0.6328328899940061


['BTC', '2022-11-17 01:46:00', 16490.00050053416]
['BTC', '2022-11-17 02:41:00', 16505.97528804